# Responsible AI Dashboard Tutorial

이 노트북은 Microsoft Responsible AI Toolbox 스타일의 종합적인 AI 모델 분석 방법을 소개합니다.

## 학습 목표
1. Responsible AI Dashboard를 사용한 모델 분석
2. Error Analysis를 통한 오류 코호트 식별
3. Counterfactual Analysis를 통한 개별 예측 설명
4. Data Balance 분석을 통한 데이터 균형 평가
5. Fairness Assessment를 통한 공정성 평가

## 참고 자료
- [Microsoft Responsible AI Toolbox](https://github.com/microsoft/responsible-ai-toolbox)
- [ResponsibleAI Dashboard](https://responsibleaitoolbox.ai/)


## 1. 환경 설정


In [ ]:
# 필요한 라이브러리 설치 (필요시)
# !pip install scikit-learn pandas numpy fairlearn shap

import sys
sys.path.insert(0, '../responsible_ai_automation')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("환경 설정 완료!")


## 2. 데이터 준비

Adult Census Income 데이터셋을 사용하여 소득 예측 모델을 구축합니다.
이 데이터셋은 공정성 분석에 자주 사용되는 벤치마크 데이터셋입니다.


In [ ]:
# Adult Census Income 데이터셋 생성 (시뮬레이션)
np.random.seed(42)
n_samples = 1000

# 특성 생성
data = {
    'age': np.random.randint(18, 70, n_samples),
    'education_years': np.random.randint(8, 20, n_samples),
    'hours_per_week': np.random.randint(20, 60, n_samples),
    'capital_gain': np.random.exponential(1000, n_samples).astype(int),
    'capital_loss': np.random.exponential(100, n_samples).astype(int),
    'gender': np.random.choice(['Male', 'Female'], n_samples, p=[0.6, 0.4]),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Other'], n_samples, p=[0.7, 0.15, 0.1, 0.05]),
    'occupation': np.random.choice(['Tech', 'Sales', 'Service', 'Admin', 'Manual'], n_samples),
}

df = pd.DataFrame(data)

# 소득 레이블 생성 (일부 편향 포함)
income_prob = (
    0.3 + 
    0.02 * (df['education_years'] - 12) +
    0.01 * (df['age'] - 30) / 10 +
    0.001 * df['capital_gain'] / 1000 +
    0.1 * (df['gender'] == 'Male').astype(int) +  # 의도적 편향
    np.random.normal(0, 0.1, n_samples)
)
income_prob = np.clip(income_prob, 0.1, 0.9)
df['income'] = (np.random.random(n_samples) < income_prob).astype(int)

print(f"데이터셋 크기: {len(df)}")
print(f"\n특성 목록: {list(df.columns)}")
print(f"\n레이블 분포:\n{df['income'].value_counts()}")
df.head()


## 3. 모델 학습


In [ ]:
# 범주형 변수 인코딩
categorical_features = ['gender', 'race', 'occupation']
sensitive_features = ['gender', 'race']

df_encoded = df.copy()
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 특성과 레이블 분리
X = df_encoded.drop('income', axis=1)
y = df_encoded['income'].values

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 모델 학습
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# 기본 성능 확인
train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)

print(f"학습 정확도: {train_acc:.4f}")
print(f"테스트 정확도: {test_acc:.4f}")


## 4. Responsible AI Dashboard 생성

Microsoft Responsible AI Toolbox 스타일의 종합 대시보드를 생성합니다.


In [ ]:
from src.dashboard import ResponsibleAIDashboard, DashboardConfig

# 대시보드 설정
config = DashboardConfig(
    title="소득 예측 모델 분석 대시보드",
    components=[
        "model_overview",
        "error_analysis",
        "fairness_assessment",
        "data_explorer",
        "counterfactual_analysis",
    ],
    theme="light",
    locale="ko",
)

# 대시보드 생성
dashboard = ResponsibleAIDashboard(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    task_type="classification",
    sensitive_features=sensitive_features,
    categorical_features=categorical_features,
    config=config,
)

print("Responsible AI Dashboard 생성 완료!")


## 5. 모델 개요 분석


In [ ]:
# 모델 개요 계산
model_overview = dashboard.compute_model_overview()

print("=" * 50)
print("모델 개요")
print("=" * 50)
print(f"태스크 유형: {model_overview['task_type']}")
print(f"학습 샘플 수: {model_overview['train_samples']:,}")
print(f"테스트 샘플 수: {model_overview['test_samples']:,}")
print(f"특성 수: {model_overview['num_features']}")
print(f"과적합 위험: {model_overview['overfitting_risk']}")

print("\n테스트 성능 메트릭:")
for metric, value in model_overview['metrics'].items():
    if metric != 'confusion_matrix':
        print(f"  - {metric}: {value:.4f}")


## 6. Error Analysis (오류 분석)

Microsoft Responsible AI Toolbox의 핵심 기능인 Error Analysis를 수행합니다.
이를 통해 모델이 특히 성능이 낮은 데이터 코호트를 식별할 수 있습니다.


In [ ]:
# 오류 분석 수행
error_analysis = dashboard.compute_error_analysis()

print("=" * 50)
print("Error Analysis 결과")
print("=" * 50)
print(f"전체 오류율: {error_analysis['overall_error_rate']:.2%}")
print(f"총 샘플 수: {error_analysis['total_samples']:,}")
print(f"총 오류 수: {error_analysis['total_errors']:,}")

print("\n식별된 오류 코호트 (상위 5개):")
for i, cohort in enumerate(error_analysis['cohorts'][:5], 1):
    print(f"  {i}. {cohort['name']}")
    print(f"     - 오류율: {cohort['error_rate']:.2%}")
    print(f"     - 샘플 수: {cohort['size']:,}")
    print(f"     - 커버리지: {cohort['coverage']:.2%}")


## 7. Data Balance 분석

데이터의 균형 상태를 분석하여 잠재적인 편향을 식별합니다.


In [ ]:
# 데이터 균형 분석
data_balance = dashboard.compute_data_balance()

print("=" * 50)
print("Data Balance 분석 결과")
print("=" * 50)

# 데이터셋 정보
info = data_balance['dataset_info']
print(f"샘플 수: {info['num_samples']:,}")
print(f"특성 수: {info['num_features']}")

# 레이블 균형
if 'label_balance' in data_balance:
    lb = data_balance['label_balance']
    print(f"\n레이블 균형:")
    print(f"  - 불균형 비율: {lb['imbalance_ratio']:.2f}:1")
    print(f"  - 균형 상태: {'✓ 균형' if lb['is_balanced'] else '✗ 불균형'}")

# 민감한 속성 균형
if 'sensitive_attribute_balance' in data_balance:
    print("\n민감한 속성별 균형:")
    for sb in data_balance['sensitive_attribute_balance']:
        print(f"\n  {sb['attribute_name']}:")
        print(f"    - Statistical Parity Difference: {sb['statistical_parity_difference']:.4f}")
        print(f"    - Disparate Impact Ratio: {sb['disparate_impact_ratio']:.4f}")
        print(f"    - 공정성: {'✓ 공정' if sb['is_fair'] else '✗ 불공정'}")


## 8. 종합 리포트 생성


In [ ]:
# 종합 리포트 생성
report = dashboard.generate_report()
print(report)


## 요약

이 튜토리얼에서 다룬 내용:

1. **ResponsibleAIDashboard**: Microsoft RAI Toolbox 스타일의 통합 대시보드
2. **Error Analysis**: 모델 오류 분석 및 문제가 되는 데이터 코호트 식별
3. **Data Balance**: 데이터 균형 및 잠재적 편향 분석
4. **Fairness Assessment**: 민감한 속성에 대한 공정성 평가

### 참고 자료
- [Microsoft Responsible AI Toolbox](https://github.com/microsoft/responsible-ai-toolbox)
- [ResponsibleAI Dashboard](https://responsibleaitoolbox.ai/)
